
# <p style="padding:10px;background-color:#87CEEB ;margin:10;color:#000000;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 10px 10px ;overflow:hidden;font-weight:50">2.  🧹 Data Preprocessing

This notebook performs comprehensive data preprocessing on the e-commerce recommendation dataset.

## Objectives and summary:
1. Load the raw dataset and standardize column names
2. Clean data fields: trim strings, lowercase text, and format dates
3. Handle missing Event_Date:
   - Identify rows with missing dates
   - Summarize total missing rows, unique users, and products
   - Optionally summarize by event type
   - Fill missing dates with median date or forward-fill to preserve patterns
4. Remove invalid or redundant columns (e.g., index)
5. Segment users into warm (2+ interactions) and cold (1 interaction)
6. Clean product names:
   - Remove punctuation, numbers, and stop words
   - Normalize units (e.g., ML → مل, KG → كيلو)
   - Tokenize and generate cleaned word lists
7. Count units in product names and generate unit frequency summary
8. Detect product name language (Arabic, English, Mixed)
9. Apply normalized and cleaned text in-place for further analysis
10. Generate summaries for user segmentation, product vocabulary, and language distribution
11. Encode IDs for modeling:
    - Convert `customer_id` → `user_idx` and `product_id` → `product_idx`
    - Create mapping dictionaries for IDs and indices
    - Store the number of unique users and products
    - Essential for matrix-based or embedding-based recommendation models
12. Robust user segmentation:
    - Identify warm and cold users based on configurable threshold (`warm_user_threshold`)
    - Returns actual sets of warm and cold users for downstream analysis


In [16]:

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# For fast similarity search
import faiss

# For models
from implicit.als import AlternatingLeastSquares


RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("✅ All libraries loaded!")


✅ All libraries loaded!


In [21]:
# ============================================================
# CELL 2: LOAD DATA (NO FILTERING - KEEP ALL USERS!)
# ============================================================
DATA_PATH = "..\\data\\raw\\csv_for_case_study_V1.csv"


df = pd.read_csv(DATA_PATH)
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')

# Clean
df['event'] = df['event'].str.lower().str.strip()
if 'index' in df.columns:
    df = df.drop('index', axis=1)

print("=" * 70)
print("📂 DATA LOADED (NO FILTERING)")
print("=" * 70)
print(f"   Total Interactions: {len(df):,}")
print(f"   Total Users: {df['customer_id'].nunique():,}")
print(f"   Total Products: {df['product_id'].nunique():,}")

# Identify warm vs cold users
user_counts = df.groupby('customer_id').size()
warm_users = set(user_counts[user_counts >= 2].index)
cold_users = set(user_counts[user_counts == 1].index)

print(f"\n📊 User Segmentation:")
print(f"   Warm users (2+ interactions): {len(warm_users):,} ({len(warm_users)/len(user_counts)*100:.1f}%)")
print(f"   Cold users (1 interaction): {len(cold_users):,} ({len(cold_users)/len(user_counts)*100:.1f}%)")
print(f"    {len(user_counts):,} users!")

📂 DATA LOADED (NO FILTERING)
   Total Interactions: 500,000
   Total Users: 433,787
   Total Products: 200,325

📊 User Segmentation:
   Warm users (2+ interactions): 49,359 (11.4%)
   Cold users (1 interaction): 384,428 (88.6%)
    433,787 users!


In [18]:
df.columns

Index(['product_id', 'customer_id', 'product_name', 'event_date', 'event'], dtype='object')

In [22]:
df.isnull().sum()

product_id          0
customer_id         0
product_name        0
event_date      22179
event               0
dtype: int64

In [23]:
# Filter rows with missing Event_Date
missing_event_df = df[df['event_date'].isnull()]

# Basic summary
total_rows_missing = len(missing_event_df)
unique_users_missing = missing_event_df['customer_id'].nunique()
unique_products_missing = missing_event_df['product_id'].nunique()
total_interactions_missing = missing_event_df.shape[0]

print(f"Total rows with missing Event_Date: {total_rows_missing}")
print(f"Unique users: {unique_users_missing}")
print(f"Unique products: {unique_products_missing}")
print(f"Total interactions (events) missing date: {total_interactions_missing}")

# Optional: summary by event type
summary_by_event = (
    missing_event_df.groupby('event')
    .agg(
        interactions=('event', 'count'),
        unique_users=('customer_id', 'nunique'),
        unique_products=('product_id', 'nunique')
    )
    .sort_values('interactions', ascending=False)
)

summary_by_event


Total rows with missing Event_Date: 22179
Unique users: 20901
Unique products: 17011
Total interactions (events) missing date: 22179


,interactions,unique_users,unique_products
event,,,
wishlist,22179,20901,17011


In [ ]:
df = df.ffill()

This step addresses missing values in the dataset:

- Most of the data is already near the **median**  
- Using **forward-fill (`ffill()`)** preserves the **original pattern** of interactions  
- Maintains the **normal distribution** of the data  
- Avoids distorting sequential trends or user/product behavior  
- Ensures data integrity for downstream analysis and modeling

In [25]:
from collections import Counter

In [26]:
df.columns

Index(['product_id', 'customer_id', 'product_name', 'event_date', 'event'], dtype='object')

In [27]:
all_words = " ".join(df['product_name'].astype(str)).split()

In [28]:
all_words

['سائل',
 'غسيل',
 'للملابس',
 'طبيعي',
 'عدد',
 '2',
 'عبوة',
 'بحجم',
 '1.8',
 'لتر',
 'عطر',
 '002',
 'ريجليز',
 'بوبا',
 'بوبا',
 'شريط',
 'لبان',
 'علكة',
 'بنكهة',
 'الفراولة',
 '56',
 'جم',
 'ساعة',
 'يد',
 'رجالية',
 'M34-2',
 'فوط',
 'تنظيف',
 'ميكروفايبر',
 'ريكسو',
 '-',
 '6',
 'حبات',
 'عباية',
 'كلاسيك',
 'مطرزه',
 'بالسدو',
 'الاسود',
 '(سدو',
 '47)',
 '-',
 'طلب',
 'مسبق',
 'جهاز',
 'العاب',
 'فيديو',
 'رقمي',
 'مع',
 'ازرار',
 'التحكم',
 'اللاسلكية',
 '(64G)',
 'تشقير',
 'حواجب',
 'بدون',
 'رتوش',
 'الصابون',
 'المغربي',
 'البلدي',
 'بزيت',
 'الارغان',
 'من',
 'جاردن',
 'اوليان',
 '-',
 '500',
 'جم',
 'عطر',
 '010',
 'مسكره',
 'حواجب',
 'شفافه',
 'ناي',
 'دريم',
 'اقراص',
 'قطنية',
 'نقية',
 'للتجميل',
 'جونسون',
 'وكس',
 'مساج',
 'تدليك',
 'العضلات',
 'اللاسلكي',
 'مع',
 '6',
 'سرعات',
 'و',
 '8',
 'رؤوس،',
 'اخضر',
 'مع',
 'اسود',
 'لباد',
 'الغيمة',
 '200×200',
 'ارتفاع',
 '12',
 'سم',
 'عباية',
 'كلوش',
 'بقماش',
 'ناعم',
 'على',
 'خطوط',
 'طولية',
 'كبسولات',
 'ستا

In [29]:
word_counts = Counter(all_words)

In [30]:
import pandas as pd
word_counts_df = pd.DataFrame(word_counts.items(), columns=['word', 'count'])

# Sort by count descending
word_counts_df = word_counts_df.sort_values(by='count', ascending=False)

print(word_counts_df.head(20))  # top 20 words
print(f"Total unique words: {len(word_counts_df)}")

       word  count
29        -  65937
59       من  42317
10      عطر  37643
137       +  16048
45       مع  14695
397       |  14158
303       (  12713
306       )  12479
32    عباية  11959
244      مل  11802
611     طقم  10580
108    جرام  10200
124     عرض  10089
393    كريم   9945
119     بكج   8883
275  مجموعة   8768
83     اسود   8585
259       3   8544
21     ساعة   7798
5         2   7794
Total unique words: 113400


In [31]:
# ============================================================
# CELL 2.5: CLEAN & PREPARE PRODUCT NAMES (Arabic + Units) - FUNCTIONAL
# ============================================================
import re
from collections import Counter
import pandas as pd

print("=" * 70)
print("🧼 CLEANING PRODUCT NAMES (Arabic + Units) - FUNCTIONS")
print("=" * 70)

# ------------------------------
# Arabic stopwords (expand anytime)
# ------------------------------
arabic_stopwords = {
    "من", "مع", "في", "على", "و", "الى", "عن", "هذا", "ذلك", 
    "او", "اي", "كل", "ثم", "هو", "هي"
}

# ============================================================
# Function 1: Normalize units (all variations → one standard)
# ============================================================
def normalize_units(text):
    text = str(text)
    # Normalize ML / مل
    ml_patterns = [r"\bML\b", r"\bMl\b", r"\bml\b", r"\bمل\b", r"\bملي\b", r"\bمليلتر\b"]
    for pat in ml_patterns:
        text = re.sub(pat, " مل ", text, flags=re.IGNORECASE)
    # Normalize KG / كيلو
    kg_patterns = [r"\bKG\b", r"\bKg\b", r"\bkg\b", r"\bكيلو\b", r"\bكغ\b"]
    for pat in kg_patterns:
        text = re.sub(pat, " كيلو ", text, flags=re.IGNORECASE)
    return text

# ============================================================
# Function 2: Clean product name (remove stopwords, symbols, numbers)
# ============================================================
def clean_product_name(name, stopwords=arabic_stopwords):
    name = str(name)
    # Apply unit normalization
    name = normalize_units(name)
    # Keep only Arabic, English letters, and spaces
    name = re.sub(r"[^\w\s\u0600-\u06FF]", " ", name)
    # Remove numbers
    name = re.sub(r"\d+", " ", name)
    # Normalize multiple spaces
    name = re.sub(r"\s+", " ", name).strip()
    # Tokenize
    words = name.split()
    # Remove stopwords
    words = [w for w in words if w not in stopwords]
    return words

# ============================================================
# Function 3: Count word frequencies
# ============================================================
def count_cleaned_words(df, column='product_name'):
    df['clean_words'] = df[column].astype(str).apply(clean_product_name)
    all_words = [word for words in df['clean_words'] for word in words]
    counts = Counter(all_words)
    counts_df = pd.DataFrame(counts.items(), columns=['word', 'count']).sort_values(by='count', ascending=False)
    return counts_df

# ============================================================
# Function 4: Detect units (after normalization)
# ============================================================
def detect_units(df, column='product_name'):
    series = df[column].astype(str).apply(normalize_units)
    units = ['مل', 'كيلو']  # normalized units
    unit_counts = {unit: int(series.str.count(fr"\b{unit}\b").sum()) for unit in units}
    return unit_counts

# ============================================================
# Function 5: Normalize and clean in-place (update clean_words & cleaned_text)
# ============================================================
def normalize_and_clean_simple(df, column='product_name'):
    for idx, text in df[column].astype(str).items():
        text = normalize_units(text)
        # Clean text
        text_clean = re.sub(r"[^\w\s\u0600-\u06FF]", " ", text)
        text_clean = re.sub(r"\s+", " ", text_clean).strip()
        words = [w for w in text_clean.split() if w not in arabic_stopwords]
        # Update columns in-place
        df.at[idx, 'clean_words'] = words
        df.at[idx, 'cleaned_text'] = " ".join(words)
    return df

# ============================================================
# USAGE EXAMPLES:
# ============================================================

# 1️⃣ Count cleaned words
cleaned_word_counts_df = count_cleaned_words(df)
print("\n📊 Top 20 frequent cleaned words:")
print(cleaned_word_counts_df.head(20))

# 2️⃣ Detect units
unit_counts = detect_units(df)
print("\n📦 Unit counts found:")
print(unit_counts)

# 3️⃣ Apply normalized & cleaned text in-place
df = normalize_and_clean_simple(df)
print("\n📝 Sample cleaned_text & clean_words:")
print(df[['product_name', 'cleaned_text', 'clean_words']].head(5))


🧼 CLEANING PRODUCT NAMES (Arabic + Units) - FUNCTIONS

📊 Top 20 frequent cleaned words:
       word  count
8       عطر  37643
211      مل  21133
91     جرام  12325
26    عباية  11981
542     طقم  10629
106     عرض  10369
342    كريم  10012
114    كيلو   9407
101     بكج   8983
70     اسود   8966
241  مجموعة   8806
17     ساعة   7837
348  اشتراك   7681
41        G   7367
470     لون   7315
686   ايفون   7237
34     جهاز   6797
131     شدة   6020
592     شهر   6002
121     حبة   5887

📦 Unit counts found:
{'مل': 13647, 'كيلو': 7431}

📝 Sample cleaned_text & clean_words:
                                        product_name  \
0    سائل غسيل للملابس طبيعي عدد 2 عبوة بحجم 1.8 لتر   
1                                            عطر 002   
2  ريجليز بوبا بوبا شريط لبان علكة بنكهة الفراولة...   
3                               ساعة يد رجالية M34-2   
4                فوط تنظيف ميكروفايبر ريكسو - 6 حبات   

                                        cleaned_text  \
0    سائل غسيل للملابس طبيعي عدد

In [32]:
import re

def normalize_units(text):
    # Standardize units
    text = re.sub(r"\bML\b", "مل", text, flags=re.IGNORECASE)
    text = re.sub(r"\bml\b", "مل", text, flags=re.IGNORECASE)
    text = re.sub(r"\bKG\b", "كيلو", text, flags=re.IGNORECASE)
    text = re.sub(r"\bKg\b", "كيلو", text, flags=re.IGNORECASE)
    text = re.sub(r"\bkg\b", "كيلو", text, flags=re.IGNORECASE)
    return text

def normalize_and_clean_units(df, column='product_name'):
    cleaned_texts = []
    for idx, text in df[column].astype(str).items():
        # Step 1: normalize units
        text = normalize_units(text)
        # Step 2: remove unwanted characters but keep Arabic + letters + digits
        text_clean = re.sub(r"[^\w\s\u0600-\u06FF]", " ", text)
        text_clean = re.sub(r"\s+", " ", text_clean).strip()
        # Step 3: remove stopwords
        words = [w for w in text_clean.split() if w not in arabic_stopwords]
        cleaned_texts.append(" ".join(words))
    df['cleaned_text'] = cleaned_texts
    return df


In [33]:
import pandas as pd
import re

def detect_language(text):
    text = str(text)
    # Count Arabic letters
    arabic_count = len(re.findall(r'[\u0600-\u06FF]', text))
    # Count English letters
    english_count = len(re.findall(r'[a-zA-Z]', text))
    
    if arabic_count > english_count:
        return 'Arabic'
    elif english_count > arabic_count:
        return 'English'
    else:
        return 'Mixed'

# Apply to product names
df['language'] = df['product_name'].apply(detect_language)

# Count how many products in each language
language_counts = df['language'].value_counts()
print(language_counts)


language
Arabic     464282
English     33256
Mixed        2462
Name: count, dtype: int64


In [34]:
# ============================================================
# CELL 5: CREATE WEIGHTED SCORES & ID MAPPINGS
# ============================================================
print("=" * 70)
print("⚙️ CREATING WEIGHTED SCORES")
print("=" * 70)

# Event weights
EVENT_WEIGHTS = {
    'purchased': 5.0,
    'cart': 3.0,
    'rating': 2.5,
    'wishlist': 2.0,
    'search_keyword': 1.0
}

df['event_weight'] = df['event'].map(EVENT_WEIGHTS).fillna(1.0)

# Recency weight
reference_date = df['event_date'].max()
df['days_ago'] = (reference_date - df['event_date']).dt.days
df['recency_weight'] = np.exp(-0.01 * df['days_ago'])

# Combined score
df['score'] = df['event_weight'] * df['recency_weight']

⚙️ CREATING WEIGHTED SCORES


In [35]:
# Define warm user threshold
warm_user_threshold = 2

# Count interactions per user
user_counts = df.groupby('customer_id').size()

# Identify warm and cold users
warm_users = set(user_counts[user_counts >= warm_user_threshold].index)
cold_users = set(user_counts[user_counts < warm_user_threshold].index)

print(f"Warm users (≥ {warm_user_threshold} interactions): {len(warm_users):,}")
print(f"Cold users (< {warm_user_threshold} interactions): {len(cold_users):,}")


Warm users (≥ 2 interactions): 49,359
Cold users (< 2 interactions): 384,428


In [36]:
from sklearn.preprocessing import LabelEncoder

# Encode user and product IDs
user_encoder = LabelEncoder()
product_encoder = LabelEncoder()

df['user_idx'] = user_encoder.fit_transform(df['customer_id'])
df['product_idx'] = product_encoder.fit_transform(df['product_id'])

# Create mapping dictionaries
mappings = {
    'user_id_to_idx': dict(zip(df['customer_id'], df['user_idx'])),
    'idx_to_user_id': dict(zip(df['user_idx'], df['customer_id'])),
    'product_id_to_idx': dict(zip(df['product_id'], df['product_idx'])),
    'idx_to_product_id': dict(zip(df['product_idx'], df['product_id'])),
    'n_users': df['user_idx'].nunique(),
    'n_products': df['product_idx'].nunique()
}

print(f"Total users: {mappings['n_users']:,}")
print(f"Total products: {mappings['n_products']:,}")

Total users: 433,787
Total products: 200,325
